<a href="https://colab.research.google.com/github/hsshih/Kaggle_Housing_Prices/blob/main/Housing_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read the housing prices data set
df_train = pd.read_csv('train.csv')
#df_test = pd.read_csv('test.csv')


#Apply log transformation to make the 'SalePrice' distribution normal
df_train['SalePrice'] = np.log(df_train['SalePrice'])

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)

In [ ]:
# Delete outliars 
df_train.sort_values(by = 'GrLivArea', ascending = False)[:2]
df_train = df_train.drop(df_train[df_train['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

In [ ]:
#Find columns with highest correlation with "Sales Price"
corrmat = df_train.corr()
k = 15 # number of variables to find (+1 to include 'Sales Price')
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index

target = df_train['SalePrice']
features = df_train[cols[1:]]
#print(features.columns)   # Print columns in selected features

In [ ]:
#Split data in the train and test sample
#from sklearn.model_selection import train_test_split
#feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size=0.5)

In [ ]:
## Try simple linear regression
#from sklearn import linear_model
#reg = linear_model.LinearRegression()
#reg.fit(feature_train, target_train)
#print(reg.coef_)
#print(reg.intercept_)
#print(reg.score(feature_train, target_train))
#print(reg.score(feature_test, target_test))
#predict = pd.DataFrame(reg.predict(feature_test),columns=['Prediction'])

In [ ]:
## Try KFold cross valication with linear regressgion
from sklearn import linear_model
scores = []
from sklearn.model_selection import KFold
reg = linear_model.LinearRegression()
cv = KFold(n_splits=10, random_state=None, shuffle=True)
for train_index, test_index in cv.split(features):
    # Use index.intersection to avoid having null values due to non-matching index
    feature_train, feature_test, target_train, target_test = \
    features.loc[features.index.intersection(train_index)], \
    features.loc[features.index.intersection(test_index)], \
    target.loc[features.index.intersection(train_index)], \
    target.loc[features.index.intersection(test_index)]
    reg.fit(feature_train, target_train)
    scores.append(reg.score(feature_test, target_test))
predict = pd.DataFrame(reg.predict(feature_test),columns=['Prediction'])
print(scores)
print(np.mean(scores))

In [ ]:
## Try Support Vector Regression
#from sklearn.svm import SVR
#svr_rbf = SVR(kernel='linear')
#svr_rbf.fit(feature_train, target_train)
#predict = pd.DataFrame(svr_rbf.predict(feature_test),columns=['Prediction'])
#print(svr_rbf.score(feature_test, target_test))

In [ ]:
# Plot predict vs. test
plt.scatter(np.exp(predict),np.exp(target_test))
plt.xlim(0,800000)
plt.ylim(0,800000)

In [ ]:
# Plot predict and test vs. 'GrLivArea'
plt.scatter(feature_test['GrLivArea'], np.exp(target_test),  color='black')
plt.scatter(feature_test['GrLivArea'], np.exp(predict), color='blue')
plt.ylim(0,800000)